# Imports

In [1]:
import numpy as np
import pandas as pd
from pymoo.problems.dynamic.df import DF1,DF2,DF3,DF4,DF5,DF6,DF7,DF8,DF9,DF10,DF11,DF12,DF13,DF14
from pymoo.indicators.igd import IGD

#my_dataset/reference_PF/reference_PF.npy
reference_PF = np.load('my_dataset/reference_PF/reference_PF.npy', allow_pickle=True)
#my_dataset/PS/ps_DF_10.csv
pos_df10 = pd.read_csv('my_dataset/POS/pos_DF_10.csv')

RNN_POS_DF1 = np.load('my_dataset/reference_PF/reference_PF.npy', allow_pickle=True)


#my_tau = np.load('my_dataset/my_tau/my_tau.npy', allow_pickle=True)

def my_igd(problem, pos, reference_PF, isDNSGA2=False):
    """
    This function calculates the mean IGD value of a set of POS solutions.
    
    Parameters:
    - problem : the problem object ex: DF1, DF2, DF3 ..etc
    - pos (pd.DataFrame): the solutions obtained from our Model
    - reference_PF (np.array): the reference set of POF solutions
    - isDNSGA2 (boolean): if you are using DNSGA2 outputs (POSs) or not
    
    Returns:
    float: the mean IGD value of the solutions
    
    """
    
    #problem = DF10()
    # new_df10.shape --> (500 500 10)
    data = pos.copy()
    if(isDNSGA2==True):
        
        new_df10 = []
        for i in range(0, data.shape[0],500):
            new_df10.append(data[i:i+500])
        new_df10 = np.array(new_df10)
        data = new_df10.copy()
    
    pof_every_generation = []
    #print(data.shape)
    for i in range(data.shape[0]):
        out = {}
        #print(np.isinf(data[i]).sum())
        problem._evaluate(x=np.abs(data[i]), out=out)
        
        #print(out["F"])
        # found that out["F"] have many nan values
        # replaced them with 0
        #out["F"] = np.nan_to_num(out["F"], 0)
        pof_every_generation.append( out["F"] )
        #print(np.isnan(out["F"]).sum())
        
    pof_every_generation = np.array(pof_every_generation)    
    #print(pof_every_generation.shape)


    hist_F = pof_every_generation.copy()
    # reference_PF 9 is of the DF10
    metric = IGD(reference_PF, zero_to_one=True)
    
    
    ###########
    ## tau ####
    ###########
    # kind of my_tau (same results better complexity + understandability)
    taut = 10
    nbr_generations = 500
    # generates a np.array of values from 9 to 500 with step size of 10
    # the following is the same as np.arange(9,500,10)
    # taut-1 because this way we are in the generation 
    # where we are exactly before the change occurs
    exactly_before_change_occurs = np.arange(taut-1,nbr_generations,taut)
    igd_values = []
    for i in range(len(hist_F)):# _F in hist_F:
        if i in exactly_before_change_occurs:
            #print(hist_F[i])
            current_igd = metric._do(hist_F[i])
            #print(current_igd)
            igd_values.append( current_igd )

    #print(igd_values)
    igd_values = np.array(igd_values)
    return np.mean(igd_values)

# CAll my_igd funct mean igd of DNSGA2 POS at df10 
mean_igd = my_igd(problem=DF10(), pos=pos_df10 , reference_PF=reference_PF[9], isDNSGA2=True)

print(mean_igd)

0.14638429509247372


In [2]:
# Call my_igd funct mean igd of DNSGA2 POS at df10 
mean_igd = my_igd(problem=DF10(), pos=pos_df10 , reference_PF=reference_PF[9], isDNSGA2=True)
print(mean_igd)

0.14638429509247372


# Our Models MIGDs.

In [8]:
%%time
DFs = [DF1,DF2,DF3,DF4,DF5,DF6,DF7,DF8,DF9,DF10,DF11,DF12,DF13,DF14]
mean_igd = []
for i in range(len(DFs)):
    RNN_POS_DFi = np.load('my_dataset/model_POSs/RNN/RNN_POS_reshaped_DF'+ str(i+1) +'.npy', allow_pickle=True)
    #print(RNN_POS_DFi.shape)
    #print(i)#,reference_PF[i].shape)
    mean_igd.append( my_igd( problem=DFs[i](tau=400), pos=RNN_POS_DFi , reference_PF=reference_PF[i]) )
mean_igd = np.array(mean_igd)
mean_igd

(100, 500, 10)
(100, 500, 10)
(100, 500, 10)
(100, 500, 10)
(100, 500, 10)
(100, 500, 10)
(100, 500, 10)
(100, 500, 10)
(100, 500, 10)
(100, 500, 10)
(100, 500, 10)
(100, 500, 10)
(100, 500, 10)
(100, 500, 10)
Wall time: 2.19 s


array([0.47936336, 0.24187984, 0.3147753 , 1.90138274, 0.44701426,
       5.07423805, 7.46813241, 0.08094575, 6.39947873, 1.22243185,
       0.42291579, 0.57970203, 0.75788541, 0.69556553])

# DNSGA2 MIGDs.

In [14]:
%%time
DFs = [DF1,DF2,DF3,DF4,DF5,DF6,DF7,DF8,DF9,DF10,DF11,DF12,DF13,DF14]
mean_igd = []
for i in range(len(DFs)):
    pos_dfi = pd.read_csv('my_dataset/POS/pos_DF_'+str(i+1)+'.csv')
    pos_dfi = pos_dfi.to_numpy()
    pos_dfi = np.reshape(pos_dfi , (500, 500, 10))
    #print(pos_dfi.shape)
    mean_igd.append( my_igd( problem=DFs[i](tau=400), pos=pos_dfi , reference_PF=reference_PF[i]))
mean_igd = np.array(mean_igd)
mean_igd

Wall time: 20.5 s


array([ 1.11562079,  0.77658034,  0.25667252,  1.8918338 ,  0.59546827,
       26.49674134,  5.02714047,  0.23063019,  3.03522292,  0.85440813,
        0.47129179,  0.47641749,  0.98232324,  0.81764316])